## PLANS AND TODO:

Get basis using NMF on regular season data for players who played in the playoffs. Consider removing players who took fewer than 5 shots

use this basis to characterize each player during regular season and playoffs to compare their shooting styles during each interval of time

Posssibly use KNN to determine which players play similarly, group them

Use this somehow

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import myFunctions as mf
import importlib

In [108]:
importlib.reload(mf)

<module 'myFunctions' from '/home/deeplearning/Desktop/Mine/myFunctions.py'>

In [32]:
plyrs = pickle.load(open( "allplayers.p", "rb" ))
plyrsreg = pickle.load(open("allplayersregular.p","rb"))
PIDS = [i for i in plyrs.keys()]


numpy.int64

In [83]:
tmp = plyrsreg[PIDS[0]]
print(len(tmp))
print(allshots)

378
               GRID_TYPE     GAME_ID GAME_EVENT_ID PLAYER_ID   PLAYER_NAME  \
0      Shot Chart Detail  0021700002           182     -2561    David West   
1      Shot Chart Detail  0021700002           216     -2561    David West   
2      Shot Chart Detail  0021700002           567     -2561    David West   
3      Shot Chart Detail  0021700025           180     -2561    David West   
4      Shot Chart Detail  0021700025           188     -2561    David West   
5      Shot Chart Detail  0021700025           628     -2561    David West   
6      Shot Chart Detail  0021700025           633     -2561    David West   
7      Shot Chart Detail  0021700025           656     -2561    David West   
8      Shot Chart Detail  0021700033           580     -2561    David West   
9      Shot Chart Detail  0021700033           605     -2561    David West   
10     Shot Chart Detail  0021700045           195     -2561    David West   
11     Shot Chart Detail  0021700045           601     -2561

In [80]:
# Create DataFrames will all shot data from regular season and playoffs
shots =  []
for i in PIDS:
    if(len(plyrsreg[i].index) < 100):
        continue # Skip players who did not take over 100 shots during regular season
    shots.append(plyrsreg[i])
    tmp = plyrs[i]
    tmp['PLAYER_ID'] *= -1 #to separate playoffs from regular. Easy to convert back to
    shots.append(tmp)
allshots = pd.concat(shots,ignore_index = True)


In [109]:
# Create smoothed bins of shots for all players
xedges = np.linspace(start=-300, stop=300, num=151)
yedges = np.linspace(start=-48, stop=372, num=106)
bin_edges = (xedges, yedges)

nx = xedges.size - 1
ny = yedges.size - 1

all_smooth = []

for i in enumerate(allshots.groupby('PLAYER_ID')):
    
    tmp, xedges, yedges = mf.bin_shots(i[1][1], bin_edges=(xedges, yedges), density=True, sigma=2) #i[1][1] is df
    ## vectorize and store into list
    all_smooth += [tmp.reshape(-1)]
    
X = np.vstack(all_smooth).T
p,n = X.shape
print(p,n)

TypeError: Cannot cast array data from dtype('O') to dtype('float64') according to the rule 'safe'

In [ ]:
plt.figure(figsize=[12, 11])
ax = sns.scatterplot(curry_data["LOC_X"],curry_data["LOC_Y"])
ax.set_xlim([-300,300])
ax.set_ylim([-100,500])
plt.show()